In [1]:
# Parameters
data_url = "https://osf.io/5d3be/"


In [2]:
import endomill
from nbmetalog import nbmetalog as nbm


In [3]:
nbm.print_metadata()


context: ci
hostname: b71b4dff68d5
interpreter: 3.8.12 (default, Jan 15 2022, 18:39:47)  [GCC 7.5.0]
nbcellexec: 3
nbname: phylogeny_simulation_mill
nbpath: /opt/hereditary-stratigraph-concept/binder/phylogenetic-inference/phylogeny_simulation_mill.ipynb
revision: null
session: 328195a0-f8b1-4be0-9834-9d52a25e5a6e
timestamp: 2022-03-07T16:53:20Z00:00




IPython==7.16.1
keyname==0.4.1
yaml==5.3.1
endomill==0.1.3
nbmetalog==0.2.6
re==2.2.1
ipython_genutils==0.2.0
logging==0.5.1.2
zmq==22.3.0
json==2.0.9
ipykernel==5.5.3


In [4]:
endomill.instantiate_over(
    parameter_packs=[
        {'data_url': data_url}
        for data_url in [
            # nk_ecoeaselection_seed110_pop100_mut.01_snapshot_3000.csv
            'https://osf.io/5d3be/',
            # nk_lexicaseselection_seed110_pop165_mut.01_snapshot_500.csv
            'https://osf.io/8ycq7/',
            # nk_randomselection_seed7_pop100_mut.01_snapshot_5000.csv
            'https://osf.io/ydxt7/',
            # nk_sharingselection_seed10_pop100_mut.01_snapshot_5000.csv
            'https://osf.io/cz9fk/',
            # nk_tournamentselection_seed140_pop100_mut.01_snapshot_5000.csv
            'https://osf.io/5ubn8/',
        ]
    ],
)


detected executing.endomill.ipynb file
skipping instantiate_over


In [5]:
# define papermill parameters
data_url: str


In [6]:
import anytree
from bs4 import BeautifulSoup
from collections import defaultdict
from hstrat import hstrat
from interval_search import doubling_search
from iterpop import iterpop as ip
import itertools as it
from keyname import keyname as kn
import numpy as np
import opytional as opyt
import pandas as pd
import random
import sys
from tqdm import tqdm
from urllib import request


In [7]:
random.seed(1) # ensure reproducibility
sys.setrecursionlimit(100000) # anytree exceeds default recursion limit on our data


In [8]:
nbm.print_metadata()


context: ci
hostname: b71b4dff68d5
interpreter: 3.8.12 (default, Jan 15 2022, 18:39:47)  [GCC 7.5.0]
nbcellexec: 8
nbname: phylogeny_simulation_mill
nbpath: /opt/hereditary-stratigraph-concept/binder/phylogenetic-inference/phylogeny_simulation_mill.ipynb
revision: null
session: 328195a0-f8b1-4be0-9834-9d52a25e5a6e
timestamp: 2022-03-07T16:53:21Z00:00




IPython==7.16.1
endomill==0.1.3
ipykernel==5.5.3
ipython_genutils==0.2.0
json==2.0.9
keyname==0.4.1
logging==0.5.1.2
nbmetalog==0.2.6
re==2.2.1
yaml==5.3.1
zmq==22.3.0
anytree==2.8.0
hstrat==0.2.0
iterpop==0.4.0
numpy==1.21.5
opytional==0.1.0
pandas==1.1.2


In [9]:
html = request.urlopen(data_url).read().decode('utf8')

soup = BeautifulSoup(html, 'html.parser')
title = soup.find('title')

data_filename = title.string.split()[-1]
print(data_filename)


nk_ecoeaselection_seed110_pop100_mut.01_snapshot_3000.csv


In [10]:
endomill.add_instance_outpath(
    f'a=phylogeny_simulation+source={data_filename}.endomill.ipynb',
)


# Retrieve Target Phylogeny from OSF


In [11]:
target_phylogeny_df = pd.read_csv(
    f'{data_url}/download',
)

nbm.print_dataframe_synopsis(target_phylogeny_df)


digest: 31ede04444a8c22e5c81fd700f8d63385862d0125173deb539510e8df81bd6b3
manifest:
  ancestor_list: '    356#  ex., [285064]'
  depth: '            320#  ex., 318'
  destruction_time: ' 308#  ex., inf'
  id: '               492#  ex., 286082'
  num_offspring: '    11#   ex., 0'
  num_orgs: '         18#   ex., 1'
  origin_time: '      337#  ex., 3000'
  phenotype: '        276#  ex., [ 0.212209 0.40814 0.0984858 0.442382 0.480717 0.336601
    0.00856907 0.39548 0.193967 0.12958 0.0293499 0.853584 0.254936 0.949996 0.188853
    0.345937 0.113471 0.399652 0.499309 0.796683 ]'
  tot_orgs: '         279#  ex., 1'
  total_offspring: '  16#   ex., 0'
num cols: 10
num cols all na: 0
num cols any na: 0
num na: 0
num rows: 492
num rows all na: 0
num rows any na: 0
size: 177K



# Create a Tree with Target Phylogeny Structure


In [12]:
# map id to anytree node
nodes = defaultdict(anytree.AnyNode)
for __, row in target_phylogeny_df.iterrows():
    node = nodes[row['id']]
    node.id = row['id']
    node.origin_time = row['origin_time']
    if 'NONE' not in row['ancestor_list']:
        ancestor_id = ip.popsingleton(
            eval(row['ancestor_list'])
        )
        node.parent = nodes[ancestor_id]

roots = {node.root for node in nodes.values()}


In [13]:
root = ip.popsingleton(roots)
root.height


319

In [14]:
max_origin_time = target_phylogeny_df['origin_time'].max()
max_origin_time


3000

In [15]:
mean_leaf_origin_time = np.mean([
    leaf.origin_time
    for leaf in root.leaves
])
mean_leaf_origin_time


2999.5328467153286

# Pick Parameters for Hereditary Stratigraphic Columns


In [16]:
def make_conditions(num_generations: int) -> pd.DataFrame:
    res = []
    for condemner_factory, target_column_bits, differentia_bit_width in it.product(
        [
            hstrat.StratumRetentionCondemnerTaperedDepthProportionalResolution,
            hstrat.StratumRetentionCondemnerRecencyProportionalResolution,
        ],
        [
            64,
            64 * 4,
            64 * 16,
        ],
        [
            1,
            8,
            64,
        ],
    ):
        policy_param = doubling_search(
            lambda x: \
                condemner_factory(x + 1).CalcNumStrataRetainedExact(num_generations)
                * differentia_bit_width > target_column_bits or x >= num_generations,
            {
                hstrat.StratumRetentionCondemnerTaperedDepthProportionalResolution: 1,
                hstrat.StratumRetentionCondemnerRecencyProportionalResolution: 0,
            }[condemner_factory],
        )

        actual_column_strata = condemner_factory(policy_param).CalcNumStrataRetainedExact(num_generations)
        actual_column_bits = actual_column_strata * differentia_bit_width

        res.append({
            'Retention Policy' : condemner_factory.__name__[25:],
            'Differentia Bit Width' : differentia_bit_width,
            'Retention Policy Resolution Parameter' : policy_param,
            'Target Retained Bits' : target_column_bits,
            'Actual Retained Bits' : actual_column_bits,
            'Retained Bits Error' : actual_column_bits - target_column_bits,
            'Actual Retained Strata' : actual_column_strata,
            'condemner' : condemner_factory(policy_param),
        })
    return pd.DataFrame.from_records(res)


In [17]:
conditions_df = make_conditions(int(mean_leaf_origin_time))
conditions_df.drop('condemner', axis=1)


,Retention Policy,Differentia Bit Width,Retention Policy Resolution Parameter,Target Retained Bits,Actual Retained Bits,Retained Bits Error,Actual Retained Strata
0,TaperedDepthProportionalResolution,1,31,64,63,-1,63
1,TaperedDepthProportionalResolution,8,3,64,56,-8,7
2,TaperedDepthProportionalResolution,64,1,64,192,128,3
3,TaperedDepthProportionalResolution,1,127,256,255,-1,255
4,TaperedDepthProportionalResolution,8,15,256,248,-8,31
5,TaperedDepthProportionalResolution,64,1,256,192,-64,3
6,TaperedDepthProportionalResolution,1,512,1024,1024,0,1024
7,TaperedDepthProportionalResolution,8,63,1024,1016,-8,127
8,TaperedDepthProportionalResolution,64,7,1024,960,-64,15
9,RecencyProportionalResolution,1,5,64,57,-7,57


# Set Up Ancestor Column


In [18]:
bundle = hstrat.HereditaryStratigraphicColumnBundle({
    kn.pack({
        'differentia' : row['Differentia Bit Width'],
        'policy' : row['Retention Policy'],
        'resolution' : row['Retention Policy Resolution Parameter'],
        'target_bits' : row['Target Retained Bits'],
        'actual_bits' : row['Actual Retained Bits'],
        'bits_error' : row['Retained Bits Error'],
        'actual_strata' : row['Actual Retained Strata'],
    }) \
        : hstrat.HereditaryStratigraphicColumn(
            stratum_differentia_bit_width=row['Differentia Bit Width'],
            stratum_retention_condemner=row['condemner'],
    )
    for __, row in conditions_df.iterrows()
})


# Simulate Inheritance of Ancestor Column Down Phylogenetic Tree


In [19]:
root = ip.popsingleton(roots)
root.hstrat_column = bundle

for node in anytree.LevelOrderIter(ip.popsingleton(roots)):
    parent = node.parent
    if parent is not None:
        node.hstrat_column = parent.hstrat_column.Clone()
        for __ in range(node.origin_time - parent.origin_time): node.hstrat_column.DepositStratum()


# Extract Pairwise MRCA Estimates for Extant Organisms


In [20]:
# impl -> mean retained bits
mean_retained_bits = {
    impl: np.mean([
        node.hstrat_column.GetNumStrataRetained()[impl]
        * int(kn.unpack(impl)['differentia'])
        for node in root.leaves
    ])
    for impl in bundle
}

records = [
    {
        **{
            'Mean Actual Retained Bits': v,
        },
        **kn.unpack(k)
    }
    for k, v in mean_retained_bits.items()
]
actual_retained_bits_df = pd.DataFrame.from_records(records)
actual_retained_bits_df.to_csv(
    f'a=actual_retained_bits+source={data_filename}',
)

actual_retained_bits_df


,Mean Actual Retained Bits,actual_bits,actual_strata,bits_error,differentia,policy,resolution,target_bits,_
0,63.000000,63,63,-1,1,TaperedDepthProportionalResolution,31,64,actual_bits=63+actual_strata=63+bits_error=-1+...
1,56.000000,56,7,-8,8,TaperedDepthProportionalResolution,3,64,actual_bits=56+actual_strata=7+bits_error=-8+d...
2,192.000000,192,3,128,64,TaperedDepthProportionalResolution,1,64,actual_bits=192+actual_strata=3+bits_error=128...
3,254.240876,255,255,-1,1,TaperedDepthProportionalResolution,127,256,actual_bits=255+actual_strata=255+bits_error=-...
4,248.000000,248,31,-8,8,TaperedDepthProportionalResolution,15,256,actual_bits=248+actual_strata=31+bits_error=-8...
5,192.000000,192,3,-64,64,TaperedDepthProportionalResolution,1,256,actual_bits=192+actual_strata=3+bits_error=-64...
6,1024.175182,1024,1024,0,1,TaperedDepthProportionalResolution,512,1024,actual_bits=1024+actual_strata=1024+bits_error...
7,1015.941606,1016,127,-8,8,TaperedDepthProportionalResolution,63,1024,actual_bits=1016+actual_strata=127+bits_error=...
8,960.000000,960,15,-64,64,TaperedDepthProportionalResolution,7,1024,actual_bits=960+actual_strata=15+bits_error=-6...
9,56.364964,57,57,-7,1,RecencyProportionalResolution,5,64,actual_bits=57+actual_strata=57+bits_error=-7+...


In [21]:
res = []
for extant1, extant2 in tqdm([*it.product(root.leaves, root.leaves)]):
    if extant1 != extant2:
        bounds = extant1.hstrat_column.CalcRankOfMrcaBoundsWith(extant2.hstrat_column)
        for impl in extant1.hstrat_column:
            res.append({
                'Column Configuration' \
                    : impl,
                'Differentia Bit Width' \
                    : kn.unpack(impl)['differentia'],
                'Stratum Retention Policy' \
                    : kn.unpack(impl)['policy'],
                'Stratum Retention Policy Resolution Parameter' \
                    : kn.unpack(impl)['resolution'],
                'Stratigraphic Column Expected Retained Bits' \
                    : kn.unpack(impl)['actual_bits'],
                'Stratigraphic Column Mean Actual Retained Bits' \
                    : mean_retained_bits[impl],
                'Stratigraphic Column Target Retained Bits' \
                    : kn.unpack(impl)['target_bits'],
                'Stratigraphic Column Expected Retained Bits Error' \
                    : kn.unpack(impl)['bits_error'],
                'Stratigraphic Column Actual Num Retained Strata' \
                    : kn.unpack(impl)['actual_strata'],
                'Taxon Compared From' \
                    : extant1.id,
                'Taxon Compared To' \
                    : extant2.id,
                'Generation of Taxon Compared From' \
                    : extant1.hstrat_column.GetNumStrataDeposited(),
                'Generation of Taxon Compared To' \
                    : extant2.hstrat_column.GetNumStrataDeposited(),
                'Generation Of MRCA Lower Bound (inclusive)' \
                    : opyt.apply_if(
                        bounds[impl],
                        lambda x: x[0],
                    ),
                'Generation Of MRCA Upper Bound (exclusive)' \
                    : opyt.apply_if(
                        bounds[impl],
                        lambda x: x[1],
                    ),
                'MRCA Bound Confidence' \
                    : extant1.hstrat_column[impl].CalcRankOfMrcaBoundsWithProvidedConfidenceLevel(),
                'Rank of Earliest Detectable Mrca With' \
                    : extant1.hstrat_column[impl].CalcRankOfEarliestDetectableMrcaWith(extant2.hstrat_column[impl]),
            })

res_df = pd.DataFrame.from_records(res)


  0%|          | 0/18769 [00:00<?, ?it/s]

  0%|          | 5/18769 [00:00<06:53, 45.42it/s]

  0%|          | 9/18769 [00:00<07:18, 42.82it/s]

  0%|          | 13/18769 [00:00<07:34, 41.25it/s]

  0%|          | 17/18769 [00:00<08:02, 38.83it/s]

...

100%|█████████▉| 18749/18769 [08:03<00:00, 39.54it/s]

100%|█████████▉| 18754/18769 [08:03<00:00, 39.95it/s]

100%|█████████▉| 18759/18769 [08:03<00:00, 39.80it/s]

100%|█████████▉| 18764/18769 [08:03<00:00, 40.14it/s]

100%|██████████| 18769/18769 [08:03<00:00, 38.78it/s]

In [22]:
res_df


,Column Configuration,Differentia Bit Width,Stratum Retention Policy,Stratum Retention Policy Resolution Parameter,Stratigraphic Column Expected Retained Bits,Stratigraphic Column Mean Actual Retained Bits,Stratigraphic Column Target Retained Bits,Stratigraphic Column Expected Retained Bits Error,Stratigraphic Column Actual Num Retained Strata,Taxon Compared From,Taxon Compared To,Generation of Taxon Compared From,Generation of Taxon Compared To,Generation Of MRCA Lower Bound (inclusive),Generation Of MRCA Upper Bound (exclusive),MRCA Bound Confidence,Rank of Earliest Detectable Mrca With
0,actual_bits=63+actual_strata=63+bits_error=-1+...,1,TaperedDepthProportionalResolution,31,63,63.000000,64,-1,63,286082,286081,3001,3001,2880,3001,0.968750,256
1,actual_bits=56+actual_strata=7+bits_error=-8+d...,8,TaperedDepthProportionalResolution,3,56,56.000000,64,-8,7,286082,286081,3001,3001,2560,3000,0.996094,0
2,actual_bits=192+actual_strata=3+bits_error=128...,64,TaperedDepthProportionalResolution,1,192,192.000000,64,128,3,286082,286081,3001,3001,2048,3000,1.000000,0
3,actual_bits=255+actual_strata=255+bits_error=-...,1,TaperedDepthProportionalResolution,127,255,254.240876,256,-1,255,286082,286081,3001,3001,2952,2992,0.968750,64
4,actual_bits=248+actual_strata=31+bits_error=-8...,8,TaperedDepthProportionalResolution,15,248,248.000000,256,-8,31,286082,286081,3001,3001,2944,3000,0.996094,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335371,actual_bits=240+actual_strata=30+bits_error=-1...,8,RecencyProportionalResolution,2,240,234.919708,256,-16,30,286033,285894,3001,3000,2976,2984,0.996094,0
335372,actual_bits=576+actual_strata=9+bits_error=320...,64,RecencyProportionalResolution,0,576,535.357664,256,320,9,286033,285894,3001,3000,2976,2992,1.000000,0
335373,actual_bits=1022+actual_strata=1022+bits_error...,1,RecencyProportionalResolution,215,1022,1021.364964,1024,-2,1022,286033,285894,3001,3000,2979,2984,0.968750,32
335374,actual_bits=1008+actual_strata=126+bits_error=...,8,RecencyProportionalResolution,14,1008,1002.919708,1024,-16,126,286033,285894,3001,3000,2981,2982,0.996094,0


In [23]:
nbm.print_dataframe_synopsis(res_df)


digest: 9bd8d12c265207056cb1aa8547eb73aa483fc00caa3554e64e65b03ac45070e2
manifest:
  Column Configuration: '                              18#   ex., actual_bits=63+actual_strata=63+bits_error=-1+differentia=1+policy=TaperedDepthProportionalResolution+resolution=31+target_bits=64'
  Differentia Bit Width: '                             3#    ex., 1'
  Generation Of MRCA Lower Bound (inclusive): '        52#   ex., 2880'
  Generation Of MRCA Upper Bound (exclusive): '        30#   ex., 3001'
  Generation of Taxon Compared From: '                 7#    ex., 3001'
  Generation of Taxon Compared To: '                   7#    ex., 3001'
  MRCA Bound Confidence: '                             3#    ex., 0.96875'
  Rank of Earliest Detectable Mrca With: '             6#    ex., 256'
  Stratigraphic Column Actual Num Retained Strata: '   13#   ex., 63'
  Stratigraphic Column Expected Retained Bits: '       14#   ex., 63'
  Stratigraphic Column Expected Retained Bits Error: ' 12#   ex., -1'
  Stra

# Save Pairwise MRCA Estimates to File


In [24]:
res_df.to_csv(
    f'a=pairwise_mrca_estimates+source={data_filename}.gz',
    compression='gzip',
)
res_df.to_csv(
    f'a=pairwise_mrca_estimates+source={data_filename}',
)
